<a href="https://colab.research.google.com/github/cahya-wirawan/ConvAI.id/blob/main/ConvAI_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Domain Chatbot with Indonesian GPT-2

In [1]:
!nvidia-smi

Sun Feb 20 08:27:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation of Huggingface's Transformers and Facebook's ParlAI
We need to install Transformers and ParlAI to be able to finetune GPT-2 with dialog datasets. We have to install the latest version of ParlAI from its repository, since the support for arbitrary GPT-2 models has been just added few days ago (18. Feb. 2022) https://github.com/facebookresearch/ParlAI/pull/4360


In [2]:
%%capture

!pip install transformers
!python -m pip install git+https://github.com/facebookresearch/ParlAI.git

In [3]:
import parlai
parlai.__version__

'1.5.1'

## Download the Indonesian ConvAI2 and Empathetic Dialog Dataset
There are many english chatbot datasets available publicly, but almost no Indonesian chatbot dataset. Therefore we translated two of the english datasets to Indonesian using Machine Translation model created by mas Akmal: https://huggingface.co/Wikidepia/marian-nmt-enid.

The two datasets are ConvAI2 (Persona Chat) and Empathetic Dialogues. For this demo, we will only fine tune the empathetic dialog dataset. We download and store it in the directory "/content/data"

In [4]:
!mkdir -p data
# !cd data && wget https://cloud.uncool.ai/index.php/s/Sr66KMPeAs85Si4/download/ConvAI.tgz
# !cd data && tar xvzf ConvAI.tgz
!cd data && wget https://cloud.uncool.ai/index.php/s/kCg6QZoZqNgxoww/download/empatheticdialogues.tgz
!cd data && tar xvzf empatheticdialogues.tgz


--2022-02-20 08:28:49--  https://cloud.uncool.ai/index.php/s/kCg6QZoZqNgxoww/download/empatheticdialogues.tgz
Resolving cloud.uncool.ai (cloud.uncool.ai)... 144.76.103.67
Connecting to cloud.uncool.ai (cloud.uncool.ai)|144.76.103.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27380436 (26M) [application/x-compressed]
Saving to: ‘empatheticdialogues.tgz’

empatheticdialogues 100%[===================>]  26.11M  6.46MB/s    in 4.0s    

2022-02-20 08:28:54 (6.46 MB/s) - ‘empatheticdialogues.tgz’ saved [27380436/27380436]

empatheticdialogues/
empatheticdialogues/empatheticdialogues/
empatheticdialogues/empatheticdialogues/train.csv
empatheticdialogues/empatheticdialogues/valid.csv
empatheticdialogues/empatheticdialogues/test.csv
empatheticdialogues/.built


In [5]:
!head data/empatheticdialogues/empatheticdialogues/train.csv

conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
hit:0_conv:1,1,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,1,Saya ingat akan melihat kembang api bersama sahabat saya. Ini adalah pertama kalinya kami menghabiskan waktu sendirian bersama. Meskipun ada banyak orang_comma_ kami merasa seperti satu-satunya orang di dunia.,5|5|5_2|2|5,
hit:0_conv:1,2,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,0,Apakah ini teman yang Anda cintai_comma_ atau hanya sahabat?,5|5|5_2|2|5,
hit:0_conv:1,3,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,1,Ini adalah sahabat. Aku merindukannya.,5|5|5_2|2|5,
hit:0_conv:1,4,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,0,Kemana dia

## Display Datasets Content
We can check the content of the dataset using the command line
`parlai display_data -t empathetic_dialogues --datapath data`
or inside the jupyter or a script. This is also valid for other commands such as training and evaluation.

In [6]:
from parlai.scripts.display_data import DisplayData
DisplayData.main(
  task='empathetic_dialogues',
  datapath='data',
  num_examples=10
)
# This is exactly the same as the following command line:
# parlai display_data -t empathetic_dialogues --datapath data --num-examples 10

08:29:01 | Opt:
08:29:01 |     allow_missing_init_opts: False
08:29:01 |     batchsize: 1
08:29:01 |     datapath: data
08:29:01 |     datatype: train:ordered
08:29:01 |     dict_class: None
08:29:01 |     display_add_fields: 
08:29:01 |     download_path: None
08:29:01 |     dynamic_batching: None
08:29:01 |     hide_labels: False
08:29:01 |     ignore_agent_reply: True
08:29:01 |     image_cropsize: 224
08:29:01 |     image_mode: raw
08:29:01 |     image_size: 256
08:29:01 |     init_model: None
08:29:01 |     init_opt: None
08:29:01 |     is_debug: False
08:29:01 |     loglevel: info
08:29:01 |     max_display_len: 1000
08:29:01 |     model: None
08:29:01 |     model_file: None
08:29:01 |     multitask_weights: [1]
08:29:01 |     mutators: None
08:29:01 |     num_examples: 10
08:29:01 |     override: "{'task': 'empathetic_dialogues', 'datapath': 'data', 'num_examples': 10}"
08:29:01 |     parlai_home: /usr/local/lib/python3.7/dist-packages
08:29:01 |     remove_political_convos: Fal

In [ ]:
# !parlai display_data -t convai2 --datapath data
# !parlai display_data -t empathetic_dialogues --datapath data


## Training/Finetuning the Indonesian GPT-2 Model

In [1]:
from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task='empathetic_dialogues',
    datapath='data',
    model='hugging_face/gpt2',
    # 'indonesian-nlp/gpt2' is the indonesian GPT-2 small model
    model_name='indonesian-nlp/gpt2',
    model_file='models/empathetic_dialogues_small',
    # model_name='indonesian-nlp/gpt2-medium-indonesian',
    # model_file='models/empathetic_dialogues_medium',
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-3,
    optimizer='adam',
    # optimizer='adafactor'
    # warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    # max_train_time=120,
    num_epochs=1,
    validation_every_n_epochs=0.25,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=2, fp16=False,
    #fp16_impl='mem_efficient',
    
    # speeds up validation
    # skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)

08:37:12 | WARNING: this model is in beta and the API is subject to change.
08:37:12 | building dictionary first...
08:37:12 | Overriding opt["fp16"] to False (previously: True)
08:37:12 | Using CUDA


Some weights of the model checkpoint at indonesian-nlp/gpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


08:37:25 | Total parameters: 124,441,344 (124,441,344 trainable)
08:37:25 | Loading existing model params from models/empathetic_dialogues_small
08:37:28 | Opt:
08:37:28 |     adafactor_eps: '[1e-30, 0.001]'
08:37:28 |     adam_eps: 1e-08
08:37:28 |     add_p1_after_newln: False
08:37:28 |     add_special_tokens: True
08:37:28 |     add_start_token: False
08:37:28 |     aggregate_micro: False
08:37:28 |     allow_missing_init_opts: False
08:37:28 |     batchsize: 2
08:37:28 |     beam_block_full_context: True
08:37:28 |     beam_block_list_filename: None
08:37:28 |     beam_block_ngram: -1
08:37:28 |     beam_context_block_ngram: -1
08:37:28 |     beam_delay: 30
08:37:28 |     beam_length_penalty: 0.65
08:37:28 |     beam_min_length: 1
08:37:28 |     beam_size: 1
08:37:28 |     betas: '[0.9, 0.999]'
08:37:28 |     bpe_add_prefix_space: None
08:37:28 |     bpe_debug: False
08:37:28 |     bpe_dropout: None
08:37:28 |     bpe_merge: None
08:37:28 |     bpe_vocab: None
08:37:28 |     compu

Some weights of the model checkpoint at indonesian-nlp/gpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


08:55:23 | Total parameters: 124,441,344 (124,441,344 trainable)
08:55:23 | Loading existing model params from models/empathetic_dialogues_small
08:55:25 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
08:55:28 | running eval: valid
09:00:18 | eval completed in 289.65s
09:00:18 | valid:
    accuracy  bleu-4  clen  ctpb  ctps  ctrunc  ctrunclen  exps  exs    f1  gen_n_toks  gpu_mem  llen  loss    lr  ltpb  ltps  \
           0 .003675 35.78  1081 710.5       0          0 19.86 5738 .1716       15.46    .2854 13.98 3.054 .0010 422.1 277.6   
    ltrunc  ltrunclen  ppl  token_acc  token_em  total_train_updates  tpb   tps  
         0          0 21.2      .3742         0                  868 1503 988.1

09:00:18 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: test
09:00:20 | running eval: test
09:05:08 | eval completed in 288.74s
09:05:08 | test:
    accur

({'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.003675),
  'clen': AverageMetric(35.78),
  'ctpb': GlobalAverageMetric(1081),
  'ctps': GlobalTimerMetric(710.5),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'exps': GlobalTimerMetric(19.86),
  'exs': SumMetric(5738),
  'f1': F1Metric(0.1716),
  'gen_n_toks': AverageMetric(15.46),
  'gpu_mem': GlobalAverageMetric(0.2854),
  'llen': AverageMetric(13.98),
  'loss': AverageMetric(3.054),
  'lr': GlobalAverageMetric(0.001),
  'ltpb': GlobalAverageMetric(422.1),
  'ltps': GlobalTimerMetric(277.6),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'ppl': PPLMetric(21.2),
  'token_acc': AverageMetric(0.3742),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(868),
  'tpb': GlobalAverageMetric(1503),
  'tps': GlobalTimerMetric(988.1)},
 {'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.003026),
  'clen': AverageMetric(38.48),
  'ctpb': GlobalAverageMetric(1088),
  '

### Download GPT-2 Medium Model finetuned on ConvAI2 and Empathetic Dialog datasets
For demonstration purpose. I have already finetuned Indonesian GPT-2 medium model with both datasets which can be downloaded from https://huggingface.co/cahya/persona_empathetic/blob/main/persona_empathetic.tar


In [2]:
# 
!mkdir -p models
!cd models && wget https://cdn-lfs.huggingface.co/cahya/persona_empathetic/05de188749c3ab1ae93978044116c4acc3b31efb9bc3b769a18f11219830db18 -O persona_empathetic.tar
!cd models && tar xvf persona_empathetic.tar

--2022-02-20 09:05:09--  https://cdn-lfs.huggingface.co/cahya/persona_empathetic/05de188749c3ab1ae93978044116c4acc3b31efb9bc3b769a18f11219830db18
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.224.154.121, 13.224.154.37, 13.224.154.93, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.224.154.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5703321600 (5.3G) [binary/octet-stream]
Saving to: ‘persona_empathetic.tar’

persona_empathetic. 100%[===================>]   5.31G  72.6MB/s    in 71s     

2022-02-20 09:06:21 (76.1 MB/s) - ‘persona_empathetic.tar’ saved [5703321600/5703321600]

persona_empathetic/
persona_empathetic/medium.trainstats
persona_empathetic/medium
persona_empathetic/medium.dict
persona_empathetic/medium.dict.opt
persona_empathetic/medium.opt


## Testing our finetuned model with the existing dataset

In [4]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='empathetic_dialogues',
    datapath='data',
    # model_file='models/empathetic_dialogues_small',
    model_file='models/persona_empathetic/medium',
    num_examples=15,
    skip_generation=False,
    inference='beam',
    beam_size=10,
    beam_context_block_ngram=3,
    beam_block_ngram=3,
    beam_min_length=15
)

09:10:38 | Overriding opt["task"] to empathetic_dialogues (previously: convai2,empathetic_dialogues)
09:10:38 | Overriding opt["inference"] to beam (previously: greedy)
09:10:38 | Overriding opt["beam_size"] to 10 (previously: 1)
09:10:38 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
09:10:38 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
09:10:38 | Overriding opt["beam_min_length"] to 15 (previously: 1)
09:10:38 | Using CUDA


Downloading:   0%|          | 0.00/207 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at flax-community/gpt2-medium-indonesian were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


09:11:26 | Total parameters: 354,825,216 (354,825,216 trainable)
09:11:26 | Loading existing model params from models/persona_empathetic/medium
09:11:53 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
09:11:58 | Opt:
09:11:58 |     adafactor_eps: '[1e-30, 0.001]'
09:11:58 |     adam_eps: 1e-08
09:11:58 |     add_p1_after_newln: False
09:11:58 |     add_special_tokens: True
09:11:58 |     add_start_token: True
09:11:58 |     aggregate_micro: False
09:11:58 |     allow_missing_init_opts: False
09:11:58 |     batchsize: 2
09:11:58 |     beam_block_full_context: True
09:11:58 |     beam_block_list_filename: None
09:11:58 |     beam_block_ngram: 3
09:11:58 |     beam_context_block_ngram: 3
09:11:58 |     beam_delay: 30
09:11:58 |     beam_length_penalty: 0.65
09:11:58 |     beam_min_length: 15
09:11:58 |     beam_size: 10
09:11:58 |     betas: '[0.9, 0.999]'
09:11:58 |     bpe_add_prefix_space: None
09:11:58 |     bpe_de

/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1749: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


- - - NEW EPISODE: empathetic_dialogues- - -
Hari ini, saat saya berangkat kerja di pagi hari, ban saya pecah di tengah jalan yang sibuk. Itu membuatku takut!
    labels: Apakah kamu baik-baik saja sekarang?
     model: Oh tidak! Saya senang Anda baik-baik saja. Apakah ada yang terluka?
Ya, saya baik-baik saja sekarang, tetapi dengan luka ringan.
    labels: Keren :) Apakah mobil Anda banyak rusak?
     model: Saya harap Anda segera merasa lebih baik. Saya harap Anda tidak mengalami kerusakan permanen.
- - - NEW EPISODE: empathetic_dialogues- - -
Beberapa minggu yang lalu, saya sedang berjalan melalui lorong saya, mengurus urusan saya sendiri, ketika tiba-tiba sebuah tangan mengulurkan tangan dari bawah meja dan meraih pergelangan kaki saya. Saya sangat terharu. Saya pikir saya didapat. Ternyata, itu anak saya.
    labels: Itu lucu, semoga dia tidak memberimu serangan jantung.
     model: Itu luar biasa. Saya senang Anda memiliki seseorang seperti itu dalam hidup Anda.
Saya mungkin tel

## Testing the finetuned model interactively
As mentioned above, we also can test the finetuned model programmatically or using the command line

### Script to test the model 

In [ ]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # model_file='models/empathetic_dialogues_small',
    model_file='models/persona_empathetic/medium',
    inference='beam',
    beam_size=10,
    beam_context_block_ngram=3,
    beam_block_ngram=3,
    beam_min_length=15
)

### Command line to test the model

In [2]:
!parlai interactive --model "hugging_face/gpt2" -mf "models/persona_empathetic/medium" --inference beam --beam-size 10 --beam-context-block-ngram 3 --beam-block-ngram 3 --beam-min-length 25 

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
00:39:17 | WARNING: this model is in beta and the API is subject to change.
00:39:17 | Overriding opt["inference"] to beam (previously: greedy)
00:39:17 | Overriding opt["beam_size"] to 10 (previously: 1)
00:39:17 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
00:39:17 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
00:39:17 | Overriding opt["beam_min_length"] to 25 (previously: 1)
00:39:17 | Using CUDA
00:39:22 | Gpt2: full interactive mode on.
Some weights of the model checkpoint at flax-community/gpt2-medium-indonesian were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another 

In [2]:
# Just copy models from Google drive
# !ls -lh /content/drive/MyDrive/ConvAI/models
# !cp -r /content/models /content/drive/MyDrive/ConvAI/
# !cp -r /content/drive/MyDrive/ConvAI/models /content